# OLYMPICS HISTORY
* According to historical records, the first ancient Olympic Games can be traced back to 776 BC. They were dedicated to the Olympian gods and were staged on the ancient plains of Olympia. They continued for nearly 12 centuries, until Emperor Theodosius decreed in 393 A.D. that all such "pagan cults" be banned.

### OLYMPIA
* Olympia, the site of the ancient Olympic Games, is in the western part of the Peloponnese which, according to Greek mythology, is the island of "Pelops", the founder of the Olympic Games. Imposing temples, votive buildings, elaborate shrines and ancient sporting facilities were combined in a site of unique natural and mystical beauty. Olympia functioned as a meeting place for worship and other religious and political practices as early as the 10th century B.C. The central part of Olympia was dominated by the majestic temple of Zeus, with the temple of Hera parallel to it.

### THE GAMES AND RELIGION
* The Olympic Games were closely linked to the religious festivals of the cult of Zeus, but were not an integral part of a rite. Indeed, they had a secular character and aimed to show the physical qualities and evolution of the performances acco

In [3]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import  Image
%matplotlib inline
img = np.array(Image.open(r"../input/olympic-img/352883-1440x900.jpg"))
fig = plt.figure(figsize=(15,12))
plt.imshow(img,interpolation="bilinear")
plt.axis("off")
plt.show()


IOError: [Errno 2] No such file or directory: '../input/olympic-img/352883-1440x900.jpg'

In [2]:
import pandas as pd
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
import io
import base64
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap
import folium
import folium.plugins
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff


ImportError: No module named wordcloud

# Data - overview 

In [4]:
events  = pd.read_csv(r"../input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
regions = pd.read_csv(r"../input/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")
events  = events.merge(regions[["NOC","region"]],left_on="NOC",right_on="NOC",how = "left")
host    = pd.read_csv(r"../input/olympic-host/olym.csv",encoding='windows-1252')


display ("EVENTS")
display (events.head().style.set_properties(**{"background-color" : "black",
                                               "color" : "lawngreen"}))


IOError: File ../input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv does not exist

In [5]:
#Data Manipulation

#replace gender values
events["Sex"] = events["Sex"].map({"M":"MALE","F":"FEMALE"})

#replacing strings
events["region"] =events["region"].str.replace("Boliva","Bolivia")

#Grouping similar sports
def sport_category(events) :
    
    skating   = ['Speed Skating','Figure Skating','Short Track Speed Skating']
    skiing    = [ 'Cross Country Skiing', 'Alpine Skiing','Nordic Combined',
                 'Freestyle Skiing', 'Ski Jumping','Snowboarding', 'Alpinism']
    Bobsleigh = ['Bobsleigh','Skeleton']
    aquatics  =  ['Swimming','Synchronized Swimming','Diving','Water Polo']
    volley    =  ['Volleyball','Beach Volleyball']
    gymnastic =  ['Gymnastics','Rhythmic Gymnastics', 'Trampolining']
    
    for i in skating :
        if events["Sport"] == i :
            return "Skating"
        
    for i in skiing :
        if events["Sport"] == i :
            return "Skiing"
    
    for i in Bobsleigh :
        if events["Sport"] == i :
            return "Bobsleigh"
    
    for i in aquatics :
        if events["Sport"] == i :
            return "Aquatics"
    
    for i in volley :
        if events["Sport"] == i :
            return "Volley"
        
    for i in gymnastic :
        if events["Sport"] == i :
            return "Gymnastic"
    
    else : 
        return events["Sport"]

events["Sport_category"] = events.apply(lambda events : sport_category(events) ,axis = 1)



#Creating binary columns for medals\
def gold_m(events)  :
    if events["Medal"] == "Gold" :
        return 1 
    else :
        return 0
def silver_m(events)  :
    if events["Medal"] == "Silver" :
        return 1 
    else :
        return 0
def bronze_m(events)  :
    if events["Medal"] == "Bronze" :
        return 1 
    else :
        return 0
    
events["Gold"]   = events.apply(lambda events:gold_m(events),axis = 1) 
events["Silver"] = events.apply(lambda events:silver_m(events),axis = 1) 
events["Bronze"] = events.apply(lambda events:bronze_m(events),axis = 1) 
events["Total"] = events["Gold"] + events["Silver"] +events["Bronze"]

#splitting summer and winter olympics

summer = events[events["Season"] == "Summer"]
winter = events[events["Season"] == "Winter"]

#Host cities data manipulation

host["Year"] =  host["Year"].fillna(method= "ffill")
host["Year"] = host["Year"].astype(int)
host["Country"] = host["Country"].str.replace("\xa0","")
host["Country"] = host["Country"].str.replace("Nazi Germany","Germany")
host["Country"] = host["Country"].str.replace("West Germany","Germany")
host["Summer (Olympiad)"] = host["Summer (Olympiad)"].fillna("WINTER")
host["Winter"] = host["Winter"].fillna("SUMMER")


# ##3
def lab(host) :
    if host["Summer (Olympiad)"] == "WINTER" :
        return "WINTER"
    if host["Winter"] == "SUMMER" :
        return "SUMMER"
    
host["season"] = host.apply(lambda host:lab(host),axis = 1)
host = host[['City', 'Country',  'Year', 'latitude','longitude', 'season']]


host_s = host[host["season"] == "SUMMER"]
host_w = host[host["season"] == "WINTER"]

NameError: name 'events' is not defined

## Summer Games Index

In [6]:
BLUE = '\033[94m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

##summer 
print ("Summer Games")
print ("-------------")
print(BLUE+UNDERLINE+BOLD+"Games           : " , summer["Year"].nunique())
print("Host Countries  : ",  host[host["season"] == "SUMMER"]["Country"].nunique())
print("No of Sports    : ",  summer["Sport"].nunique())
print("No of Events    : ",  summer["Event"].nunique())
s_male_ev   = summer[summer["Sex"] == "MALE"]["Event"].unique().tolist()
s_female_ev = summer[summer["Sex"] == "FEMALE"]["Event"].unique().tolist()
s_both_ev   = [x for x in s_male_ev if x in s_female_ev]
s_male_events   = [x for x in s_male_ev if x not in s_both_ev]
s_female_events = [x for x in s_female_ev if x not in s_both_ev]
print("Events for female athletes : " , len(s_female_events))
print("Events for male athletes   : " , len(s_male_events))
print("Events for both genders    : " , len(s_both_ev))
print("Total NOC's     : ",  summer["NOC"].nunique())
print("Total Participant's  : ",  summer["ID"].nunique())
print("Female Participant's : ",  summer[summer["Sex"] == "FEMALE"]["ID"].nunique())
print("Male Participant's   : ",  summer[summer["Sex"] == "MALE"]["ID"].nunique())
print("Youngest Participant : ",summer.loc[summer["Age"].idxmin()]["Name"],
      " , Age - " , (summer.loc[summer["Age"].idxmin()]["Age"]).astype(int),
      " , Country - " ,summer.loc[summer["Age"].idxmin()]["region"] ,
      " , Year - " ,summer.loc[summer["Age"].idxmin()]["Year"])
print("Oldest Participant : ",summer.loc[summer["Age"].idxmax()]["Name"],
      " , Age - " , (summer.loc[summer["Age"].idxmax()]["Age"]).astype(int),
      " , Country - " ,summer.loc[summer["Age"].idxmax()]["region"] ,
      " , Year - " ,summer.loc[summer["Age"].idxmax()]["Year"])
s_med  = summer.groupby("Name")["Total"].sum().reset_index()
print("Most Medals (Athlete) : ", s_med.loc[s_med["Total"].idxmax()]["Name"])
s_med_c = summer.groupby("region")["Total"].sum().reset_index()
print("Most Medals (Country) : ",s_med_c.loc[s_med_c["Total"].idxmax()]["region"])


Summer Games
-------------


NameError: name 'summer' is not defined

## Winter Games Index

In [ ]:

##winter
print ("Winter Games")
print ("-------------")
print(BLUE+UNDERLINE+BOLD+"Games           : " , winter["Year"].nunique())
print("Host Countries  : ",  host[host["season"] == "WINTER"]["Country"].nunique())
print("No of Sports    : ",  winter["Sport"].nunique())
print("No of Events    : ",  winter["Event"].nunique())

w_male_ev   = winter[winter["Sex"] == "MALE"]["Event"].unique().tolist()
w_female_ev = winter[winter["Sex"] == "FEMALE"]["Event"].unique().tolist()
w_both_ev   = [x for x in w_male_ev if x in w_female_ev]
w_male_events   = [x for x in w_male_ev if x not in w_both_ev]
w_female_events = [x for x in w_female_ev if x not in w_both_ev]
print("Events for female athletes : " , len(w_female_events))
print("Events for male athletes   : " , len(w_male_events))
print("Events for both genders    : " , len(w_both_ev))

print("Total NOC's     : ",  winter["NOC"].nunique())
print("Total Participant's  : ",  winter["ID"].nunique())
print("Female Participant's : ",  winter[winter["Sex"] == "FEMALE"]["ID"].nunique())
print("Male Participant's   : ",  winter[winter["Sex"] == "MALE"]["ID"].nunique())
print("Youngest Participant : ",winter.loc[winter["Age"].idxmin()]["Name"],
      " , Age - " , (winter.loc[winter["Age"].idxmin()]["Age"]).astype(int),
      " , Country - " ,winter.loc[winter["Age"].idxmin()]["region"] ,
      " , Year - " ,winter.loc[winter["Age"].idxmin()]["Year"])
print("Oldest Participant : ",winter.loc[winter["Age"].idxmax()]["Name"],
      " , Age - " , (winter.loc[winter["Age"].idxmax()]["Age"]).astype(int),
      " , Country - " ,winter.loc[winter["Age"].idxmax()]["region"] ,
      " , Year - " ,winter.loc[winter["Age"].idxmax()]["Year"])
w_med  = winter.groupby("Name")["Total"].sum().reset_index()
print("Most Medals (Athlete) : ", w_med.loc[w_med["Total"].idxmax()]["Name"])
w_med_c = winter.groupby("region")["Total"].sum().reset_index()
print("Most Medals (Country) : ",w_med_c.loc[w_med_c["Total"].idxmax()]["region"])

# Athletes participation for Summer and Winter Olympics over Years
* Summer olympics : 
Summer Olympics are the Olympic Games which are principally held in the summer season. The games and sporting events organized at the Summer Olympics are totally in contrast to the ones held at Winter Olympics. It is also fair to say that this is the competition which stages the most competitive and the most widely followed sporting events of the world. These events include the likes of athletics, football, basketball, swimming, weightlifting, hockey, volleyball, badminton, gymnastics, etc. 

* Winter Olympics : 
The Winter Olympics, as the name suggests, are held particularly in the winter season. The games played are specific to this kind of season, and are also called as winter sports. Some of the most popular winter sports such as ice-skating, figure skating, ice hockey, snowboarding, skiing, etc. gather a world audience through the Winter Olympics.

* Interestingly, both these editions were held in the same years, until the year 1992, before the IOC (International Olympic committee) decided to organize the two events at a gap of every two years. As a consequence, the next Winter Olympics was held in the year 1994, instead of being held in the year 1996.


In [ ]:
sum_c = summer.groupby(["Year"])["ID"].nunique().reset_index()
win_c = winter.groupby(["Year"])["ID"].nunique().reset_index()

fig = plt.figure(figsize=(13,16))
plt.subplot(211)
ax = sns.pointplot(x = sum_c["Year"] , y = sum_c["ID"],
                   color="white",markers="h")
plt.xticks(rotation = 60)
ax.set_facecolor("k")
plt.grid(True,alpha=.2)
plt.ylabel("count")
plt.title("Athletes over years for Summer Olympics",color="b")

plt.subplot(212)
ax1 = sns.pointplot(x = win_c["Year"] , y = win_c["ID"],
                    color = "red",markers="h")
plt.xticks(rotation = 60)
ax1.set_facecolor("k")
plt.grid(True,alpha=.2)
plt.ylabel("count")
plt.title("Athletes over years for Winter Olympics",color="b")
plt.subplots_adjust(hspace = .3)
plt.show()

# Athletes participation by gender over years for Olympics
Women participated for the first time at the 1900 Paris Games with the inclusion of women's events in lawn tennis and golf. Women's athletics and gymnastics debuted at the 1928 Olympics. Over time more women's events were added. In 2012, women's boxing was introduced, resulting in no remaining sports that do not include events for women.

In [ ]:
sum_gc = summer.groupby(["Year","Sex"])["ID"].nunique().reset_index()
win_gc = winter.groupby(["Year","Sex"])["ID"].nunique().reset_index()

fig = plt.figure(figsize=(13,16))
plt.subplot(211)
ax = sns.pointplot(x = sum_gc["Year"] , y = sum_gc["ID"],
                   markers="h" , hue = sum_gc["Sex"],palette = ["r","b"])
plt.grid(True)
plt.xticks(rotation = 60)
ax.set_facecolor("lightgrey")
plt.ylabel("count")
plt.title("Athletes by gender over years for Summer Olympics",color="b")
plt.legend(loc = "best" ,prop={"size":15})

plt.subplot(212)
ax1 = sns.pointplot(x = win_gc["Year"] , y = win_gc["ID"],
                    markers="h",palette = ["b","r"],
                   hue = win_gc["Sex"])
plt.xticks(rotation = 60)
ax1.set_facecolor("lightgrey")
plt.grid(True)
plt.ylabel("count")
plt.title("Athletes by gender over years for Winter Olympics",color="b")
plt.legend(loc = "best" ,prop={"size":15})
plt.subplots_adjust(hspace = .3)
plt.show()


# Gender Distribution for Summer and Winter Olympics
Winter Olympics have higher female percent participation compared to summer olympics

In [ ]:
plt.figure(figsize=(13,6))
plt.subplot(121)
summer.groupby("Sex")["ID"].nunique().plot.pie(autopct = "%1.0f%%",
                                               wedgeprops = {"linewidth":2,"edgecolor":"w"},
                                              explode = [0,.01],shadow = True , 
                                               colors = ["royalblue","lawngreen"]) 
plt.ylabel("")
circ = plt.Circle((0,0),.7,color = "white")
plt.gca().add_artist(circ)
plt.title("SUMMER OLYMPICS")

plt.subplot(122)
winter.groupby("Sex")["ID"].nunique().plot.pie(autopct = "%1.0f%%",
                                               wedgeprops = {"linewidth":2,"edgecolor":"w"},
                                              explode = [0,.01],shadow = True , 
                                               colors = ["royalblue","lawngreen"]) 
plt.ylabel("")
circ = plt.Circle((0,0),.7,color = "white")
plt.gca().add_artist(circ)
plt.title("WINTER OLYMPICS")

plt.show()

# Countries participation at the Olympic Games
* Nations, as represented by National Olympic Committees (NOCs), that have participated in the Summer Olympic Games between 1896 and 2016. As of the 2016 Games, all of the current  NOCs have participated in at least one edition of the Olympic Games, and athletes from Australia , France, Great Britain, Greece and Switzerland have competed in all  Summer Olympic Games.

In [ ]:
plt.figure(figsize=(12,16))
plt.subplot(211)
ax = summer.groupby("Year")["NOC"].nunique().plot(kind = "bar",
                                                 colors = sns.color_palette("husl"),
                                                 linewidth = 1,
                                                  edgecolor=  "w"*summer["Year"].nunique())
plt.xticks(rotation = 60)
ax.set_facecolor("lightgrey")
plt.ylabel("count")
plt.title("Countries participated at the Summer Olympic Games")
plt.grid(True,alpha=.3)

plt.subplot(212)
ax1 = winter.groupby("Year")["NOC"].nunique().plot(kind = "bar",
                                                 colors = sns.color_palette("husl"),
                                                 linewidth = 1,
                                                   edgecolor=  "w"*summer["Year"].nunique())
plt.xticks(rotation = 60)
ax1.set_facecolor("lightgrey")
plt.title("Countries participated at the Winter Olympic Games")
plt.grid(True,alpha=.3)
plt.show()


# Nations with highest number of participation in olympics

In [ ]:
yr_cn = pd.crosstab(summer["region"],summer["Year"]).reset_index()
cols = yr_cn.columns[1:]

for i in cols:
    yr_cn[i] = np.where(yr_cn[i] > 0 ,1,0)
    
yr_cn["total"] = yr_cn[cols].sum(axis = 1)
yr_cn = yr_cn.sort_values(by = "total",ascending=False)


plt.figure(figsize=(12,12))
plt.subplot(121)
ax = sns.barplot("total","region",data=yr_cn[:25],
                 linewidth = 1,edgecolor = "k"*25,
                 palette = "rainbow"
                )

for i,j in enumerate(yr_cn["total"][:25].astype(str) + " times"):
    ax.text(.7,i,j,fontsize = 12)
    
plt.title("countries - summer olympics")

##
yr_cn1 = pd.crosstab(winter["region"],winter["Year"]).reset_index()
cols = yr_cn1.columns[1:]

for i in cols:
    yr_cn1[i] = np.where(yr_cn1[i] > 0 ,1,0)
    
yr_cn1["total"] = yr_cn1[cols].sum(axis = 1)
yr_cn1 = yr_cn1.sort_values(by = "total",ascending=False)



plt.subplot(122)
ax = sns.barplot("total","region",data=yr_cn1[:25],
                 linewidth = 1,edgecolor = "k"*25,
                 palette = "rainbow"
                )

for i,j in enumerate(yr_cn1["total"][:25].astype(str) + " times"):
    ax.text(.7,i,j,fontsize = 12)
    
plt.title("countries - winter olympics")
plt.ylabel("")
plt.subplots_adjust(wspace = .3)
plt.show()

## Countries participation in summer olympics
*  athletes from Australia , France, Great Britain, Greece and Switzerland have competed in all  Summer Olympic Games.

In [ ]:
reg_part_s = yr_cn[["region","total"]]
trace = go.Choropleth(locations = reg_part_s["region"],
                      locationmode="country names",
                      z = reg_part_s["total"],
                      colorscale = "Picnic",
                      autocolorscale = False,
                      reversescale = True,
                      marker = dict(line = dict(width = 1,
                                                color = "black")
                                   ),
                      colorbar = dict(title = "Count of Participation",
                                      titleside = "right"
                                     ),
                     )

data = [trace]

layout = go.Layout(dict(title = "Countries participation in summer olympics",
                        geo = dict(showframe = True,
                                   showocean = True,
                                   oceancolor = "#10223A",
                                   bgcolor = "rgb(255, 204, 51)", 
                                   showcoastlines = True,
                                   projection = dict(type = 'equirectangular'),
                                  ),
                       )
                  )

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)


## Countries participation in winter olympics
* Austria, Canada, Finland, France, Great Britain, Hungary, Italy, Norway, Poland, Sweden, Switzerland, and the United States have participated in all twenty-two Winter Games to date. Including continuity from Czechoslovakia, the Czech Republic and Slovakia have also been represented in every edition.

In [ ]:
reg_part_w = yr_cn1[["region","total"]]
trace = go.Choropleth(locations = reg_part_w["region"],
                      locationmode="country names",
                      z = reg_part_w["total"],
                      text = reg_part_w["region"],
                      colorscale = "Picnic",
                      autocolorscale = False,
                      reversescale = True,
                      marker = dict(line = dict(width = 1,
                                                color = "black")
                                   ),
                      colorbar = dict(title = "Count of Participation",
                                      titleside = "right"
                                     ),
                     )

data = [trace]

layout = go.Layout(dict(title = "Countries participation in winter olympics",
                        geo = dict(showframe = True,
                                   showocean = True,
                                   showcountries = True,
                                   oceancolor = "#10223A",
                                   bgcolor = "rgb(255, 204, 51)",
                                   showcoastlines = True,
                                   projection = dict(type = "equirectangular")
                                  ),
                       )
                  )

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Nations that have hosted the Olympics
There have been 28 Summer Olympic Games held in 23 cities, and 23 Winter Olympic Games held in 20 cities. In addition, three summer and two winter editions of the Games were scheduled to take place but later cancelled due to war: Berlin (summer) in 1916; Tokyo/Helsinki (summer) and Sapporo/Garmisch-Partenkirchen (winter) in 1940; and London (summer) and Cortina d'Ampezzo, Italy (winter) in 1944. The 1906 Summer Olympics were officially sanctioned and held in Athens. However, in 1949, the International Olympic Committee (IOC), decided to unrecognize the 1906 Games

In [ ]:
hst_c_s = host_s["Country"].value_counts().reset_index()

trace = go.Choropleth(locations = hst_c_s["index"],
                       locationmode="country names",
                       z = hst_c_s["Country"],
                       colorscale = "Portland",
                       marker = dict(line = dict(width = 1,color = "black")),
                       colorbar = dict(title = "Count of hosted countries",
                                       titleside = "right"
                                      )
                     )

layout = go.Layout(dict(title = "Nations that have hosted the Summer Olympics",
                        geo = dict(showframe = False,
                                   bgcolor = "white",
                                   showocean = True,
                                   showcountries =True,
                                   showcoastlines = True,
                                   projection = dict(type = "equirectangular")
                                  ),
                       ),
                  )

data  = [trace]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)


In [ ]:
hst_c_w = host_w["Country"].value_counts().reset_index()


trace = go.Choropleth(locations = hst_c_w["index"],
                       locationmode="country names",
                       z = hst_c_w["Country"],
                       colorscale = "Portland",
                       marker = dict(line = dict(width = 1,color = "black")),
                       colorbar = dict(title = "Count of hosted countries",
                                       titleside = "right"
                                      )
                     )

layout = go.Layout(dict(title = "Nations that have hosted the Winter Olympics",
                        geo = dict(showframe = False,
                                   showcountries =True,
                                   showocean = True,
                                   showcoastlines = True,
                                   projection = dict(type = 'equirectangular')
                                  ),
                       ),
                  )

data  = [trace]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Countries that hosted highest number of olympics
* The United States leads the world by hosting 8 Olympic Games including 4 Summer Olympics and 4 Winter Olympics (1904, 1932, 1984, 1996, 1932, 1960, 1980, 2002). US Summer Olympic venues have included St. Louis, Atlanta, and two times in Los Angeles. Winter Games have been held twice at Lake Placid, and once each in Squaw Valley and Salt Lake City.
* France is second in the list, hosting the Olympics five times (1900, 1924, 1924, 1968, 1992).  France has hosted two Summer Olympics in Paris, and Winter Games in Chamonix, Grenoble, and Albertville.
* The United Kingdom is in third as a three-time Olympic host (1908, 1948, 2012).  All three Olympic Games were Summer Games, and all were hosted in the city of London.
* Japan is  3-time Olympic host (1964, 1972, 1998).  Both of these Summer Games were hosted in Tokyo, and Winter Games were hosted in Nagano and Sapporo.
* Canada is as a three-time Olympic host (1976, 1988, 2010). Montreal has hosted a Summer Games, and Vancouver and Calgary have each hosted a Winter Games venue.
* Italy is  three-time Olympic host (1960, 1956, 2006). A Summer Games was held in Rome, while Winter Games have been held in Turin and Cortina d'Ampezzo.
* Germany is three-time Olympic host (1936, 1972, 1936). Munich and Berlin have each hosted a Summer Games Venue, while the 1936 Winter Olympics were held at Garmisch-Partenkirchin.

In [ ]:
sns.set_style("darkgrid")
host_s["Year"] = host_s["Year"].astype(str)
hs = host_s.groupby("Country")["Year"].apply(" , ".join).reset_index()
hs1 = host_s["Country"].value_counts().reset_index()
h_s = hs.merge(hs1,left_on = "Country",right_on = "index",how = "left")
h_s = h_s.sort_values(by = "Country_y",ascending=False)

plt.figure(figsize=(8,10))
ax = sns.barplot("Country_y","Country_x",
                 data=h_s ,linewidth = 1,
                 palette = "summer",
                 edgecolor = "k"*h_s["Country_x"].nunique())
for i,j in enumerate("Years : " + h_s["Year"]) :
    ax.text(.1,i,j,fontsize = 12)
plt.grid(True)
plt.xticks(np.arange(0,5,1))
plt.xlabel("count")
plt.ylabel("country")
plt.title("Countries that hosted highest number of summer olympics")
plt.show()

In [ ]:
host_w["Year"] = host_w["Year"].astype(str)
hw = host_w.groupby("Country")["Year"].apply(" , ".join).reset_index()
hw1 = host_w["Country"].value_counts().reset_index()
h_w = hw.merge(hw1,left_on = "Country",right_on = "index",how = "left")
h_w = h_w.sort_values(by = "Country_y",ascending=False)

plt.figure(figsize=(8,8))
ax = sns.barplot("Country_y","Country_x",
                 data=h_w ,linewidth = 1,
                 palette = "winter",
                 edgecolor = "k"*h_w["Country_x"].nunique())
for i,j in enumerate("Years : " + h_w["Year"]) :
    ax.text(.1,i,j,fontsize = 12)
plt.grid(True)
plt.xticks(np.arange(0,5,1))
plt.xlabel("count")
plt.ylabel("country")
plt.title("Countries that hosted highest number of winter olympics")
plt.show()

## Olympic Games host cities
* Map of host cities and countries of the  summer  and winter Olympics. 

In [ ]:
cit_ll = host.groupby(["Country","latitude","longitude"])["City"].value_counts().to_frame()
cit_ll = cit_ll.rename(columns={"City" : "Count"})
cit_ll = cit_ll.reset_index().sort_values(by = "Count",ascending =False)
host["Year"] = host["Year"].astype(str)
x = host.groupby("City")["Year"].apply(",".join).reset_index()
cit_ll= cit_ll.merge(x,left_on="City",right_on="City",how ="left")
cit_ll
cit_ll

def map_city(count,name,color):
    tracer = go.Scattergeo(lat=cit_ll[cit_ll["Count"] == count]["latitude"],
                          lon=cit_ll[cit_ll["Count"] == count]["longitude"],
                          text=(cit_ll[cit_ll["Count"] == count]["City"] +" , "+ 
                                cit_ll[cit_ll["Count"] == count]["Country"]
                                +",Year : "+ cit_ll[cit_ll["Count"] == count]["Year"]),
                          mode = "markers",
                          marker = dict(size = 9,
                                        opacity = .8,
                                        line = dict(width = 1,
                                                    color = 'rgba(102, 102, 102)',
                                                   ),
                                        symbol = "diamond",
                                        reversescale = True,
                                        autocolorscale = False,
                                        color = color,
                                       ),
                           name = name,
                         )
    return tracer

layout = go.Layout(dict(title = "Olympic Games host cities",
                        geo = dict(scope = "world",
                                   showland = True,
                                   landcolor = "white",
                                   subunitcolor = "red",
                                   subunitwidth = 1,
                                   showcountries = True,
                                   countrycolor = "black",
                                   countrywidth = .1,
                                   showocean = True,
                                   projection = dict(type = "orthographic",
                                                     rotation = dict(lat = 40,lon = 25)
                                                    ),
                                  ),
                        height = 650,
                        width =  650,
                       )
                  )

trace1 = map_city(3,"THRICE","blue")
trace2 = map_city(2,"TWICE","green")
trace3 = map_city(1,"ONCE","red")

data = [trace3,trace2,trace1]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Host cities for summer and winter olympics
* London became the first city to have hosted three Games with the 2012 Summer Olympics.
* Cities hosted the Olympic Games more than once: Athens (1896 and 2004 Summer Olympics), Paris (1900, 1924 ), London (1908, 1948 and 2012 Summer Olympics), St. Moritz (1928 and 1948 Winter Olympics), Lake Placid (1932 and 1980 Winter Olympics), Los Angeles (1932, 1984), Innsbruck (1964 and 1976 Winter Olympics )

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(211)
sns.countplot(host_s["City"],facecolor = (0,0,0,0),
                  linewidth = 3,
                  edgecolor = "b"*host_s["City"].nunique())
plt.xticks(rotation = 60)
plt.grid(True)
plt.title("Host cities for summer olympics")
plt.yticks(np.arange(0,4,1))

plt.subplot(212)
sns.countplot(host_w["City"],facecolor = (0,0,0,0),
              linewidth = 3,
              edgecolor = "k"*host_w["City"].nunique())
plt.xticks(rotation = 80)
plt.grid(True)
plt.title("Host cities for winter olympics")
plt.yticks(np.arange(0,4,1))

plt.subplots_adjust(hspace = .4)

## Height and Weight Distribiution of Athletes
* Height - In centimeters
* Weight - In kilograms

In [ ]:
cols = ["Weight","Height"]
length = len(cols)
cs = ["b","r"]

plt.figure(figsize= (13,8))

for i,j,k in itertools.zip_longest(cols,range(length),cs) :
    sns.distplot(events[events[i].notnull()][i],color = k,label = i)
    plt.axvline(events[events[i].notnull()][i].mean(),
                color = k,linestyle = "dashed",label = i+ " average")
    plt.legend(loc = "best",prop = {"size" : 14})
    plt.title("Height and Weight Distribution of Athletes")
    plt.subplots_adjust(hspace = .2)
    plt.xticks(np.arange(0,250,20))

## Body mass index :
The body mass index (BMI) or Quetelet index is a value derived from the mass (weight) and height of an individual.The BMI is an attempt to quantify the amount of tissue mass (muscle, fat, and bone) in an individual . The BMI is defined as the body mass divided by the square of the body height, and is universally expressed in units of kg/m2, resulting from mass in kilograms and height in metres.. A body mass index between 18.5 and 24.9 falls in the healthy range.

In [ ]:
#Selecting non null values of height and weight
metric_data = events[(events["Height"].notnull()) & (events["Weight"].notnull())]
#convert height to meters
metric_data["Height_in_mt"] = metric_data["Height"]/100
#caluculate BMI
metric_data["BMI"] = metric_data["Weight"]/(metric_data["Height_in_mt"]**2)

plt.figure(figsize=(13,7))
sns.distplot(metric_data["BMI"],color = "k")
plt.axvline(metric_data["BMI"].mean(),
            linestyle = "dotted",linewidth = 3,
            color= "k",label = "BMI - AVERAGE")
plt.legend(loc = "best",prop = {"size" : 13})
plt.title("Distribution of Body Mass Index of Athletes")
plt.show()

## Height and weight of atheletes w.r.t BMI

In [ ]:
plt.figure(figsize=(13,9))
sns.set_style("darkgrid")
plt.scatter(metric_data["Height"],metric_data["Weight"],cmap="jet",
            c=metric_data["BMI"],linewidth=.4,edgecolor = "k")
lab = plt.colorbar()
lab.set_label("Body Mass Index",fontsize = 13)
plt.xlabel("Height_in_centimeters")
plt.ylabel("Weight in kilograms")
plt.title("Height and weight of atheletes w.r.t BMI",color = "navy")
plt.show()

## 3d scatter plot between height,weight and age 

In [ ]:
#metric_data.isnull().sum()
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("whitegrid")

fig = plt.figure(figsize=(13,10))
ax  = fig.gca(projection = "3d")
plot = ax.scatter(metric_data[metric_data["Age"].notnull()]["Height_in_mt"],
                  metric_data[metric_data["Age"].notnull()]["Weight"],
                  metric_data[metric_data["Age"].notnull()]["Age"],
                  c = metric_data[metric_data["Age"].notnull()]["BMI"],
                  cmap = "jet",
                  linewidth = .1,
                  edgecolor = "k",
                  s = 60,
                 )
lab = fig.colorbar(plot,shrink = .3,aspect = 5)
lab.set_label("BMI")
ax.set_xlabel("Height")
ax.set_ylabel("Weight")
ax.set_zlabel("Age")
plt.title("3d scatter plot for height,weight and age " ,color = "navy")
plt.show()

## Hexbin plot for (Height,Weight,Age) & BMI

In [ ]:
plt.figure(figsize=(9,20))
cols = ["Age","Weight","Height_in_mt"]
length = len(cols)

for i,j in itertools.zip_longest(cols,range(length)):
    plt.subplot(3,1,j+1)
    plt.hexbin(metric_data[i],metric_data["BMI"],
               gridsize=(40,40),cmap = "hot") 
    plt.xlabel(i)
    plt.ylabel("BMI")
    plt.colorbar(label = "data points")
    plt.title("BMI Vs " +  i)

## Average of Attributes by Sport Category

In [ ]:
cols = ['Age', 'Weight' ,'Height',"BMI"]
length = len(cols)
palette = ["b","r","grey","lime"]

sns.set_style("darkgrid")
plt.figure(figsize=(11,25))
for i,j,k in itertools.zip_longest(cols,range(length),palette) :
    plt.subplot(4,1,j+1)
    avg = metric_data.groupby("Sport_category")[i].mean().reset_index()
    avg = avg.sort_values(by = i ,ascending =False)
    sns.barplot("Sport_category",i,data=avg[avg[i].notnull()],
                linewidth = .5,edgecolor = "k"*len(avg),
                color = k)
    plt.xticks(rotation = 90)
    plt.subplots_adjust(hspace = .6)
    plt.grid(True)
    plt.title("Average "+ i + "  by Sports category")
    plt.xlabel("")

# Events for olympics 
* Events include categories in each sport for male and female genders

In [ ]:
ev_s = summer.groupby("Year")["Event"].nunique().reset_index()
ev_w = winter.groupby("Year")["Event"].nunique().reset_index()

plt.figure(figsize=(13,14))
plt.subplot(211)
ax = sns.pointplot(ev_s["Year"],ev_s["Event"],color="w",markers="H")
plt.xticks(rotation = 60)
ax.set_facecolor("k")
plt.grid(True,alpha=.1)
plt.ylabel("count")
plt.title("Unique sport Events over years for Summer Olympics",color="b")

plt.subplot(212)
ax = sns.pointplot(ev_w["Year"],ev_w["Event"],color="r",markers="H")
plt.xticks(rotation = 60)
ax.set_facecolor("k")
plt.grid(True,alpha=.1)
plt.ylabel("count")
plt.title("Unique sport Events over years for Winter Olympics",color="b")
plt.subplots_adjust(hspace = .3)
plt.show()

## Events ratio by gender

In [ ]:
plt.figure(figsize=(13,6))
plt.subplot(121)
summer.groupby("Sex")["Event"].nunique().plot.pie(autopct = "%1.0f%%",
                                                  wedgeprops = {"linewidth":2,"edgecolor":"w"},
                                                  explode = [0,.01],shadow = True , 
                                                  colors = ["royalblue","lawngreen"])
plt.title("Events ratio by gender in summer olympics",color = "r")
plt.ylabel("")

plt.subplot(122)
winter.groupby("Sex")["Event"].nunique().plot.pie(autopct = "%1.0f%%",
                                                  wedgeprops = {"linewidth":2,"edgecolor":"w"},
                                                  explode = [0,.01],shadow = True , 
                                                  colors = ["royalblue","lawngreen"])
plt.title("Events ratio by gender in winter olympics",color = "r")
plt.ylabel("")

plt.show()

## Events by gender over years
* The events of women in the Olympics has been increasing since their first participation in 1900. 

In [ ]:
sns.set_style("darkgrid")
s_ys = summer.groupby(["Year","Sex"])["Event"].nunique().reset_index()

plt.figure(figsize=(13,13))
plt.subplot(211)
sns.barplot("Year","Event",
            data=s_ys,hue="Sex",
            linewidth = 1,palette = ["b","r"],
            edgecolor = "k"*s_ys["Year"].nunique()
           )
plt.xticks(rotation = 60)
plt.grid(True)
plt.legend(loc = "upper left",prop = {"size" : 15})
plt.title("Events by gender for summer olympics")

w_ys = winter.groupby(["Year","Sex"])["Event"].nunique().reset_index()

plt.figure(figsize=(13,13))
plt.subplot(212)
sns.barplot("Year","Event",
            data=w_ys,hue="Sex",
            linewidth = 1,palette = ["b","r"],
            edgecolor = "k"*w_ys["Year"].nunique()
           )
plt.xticks(rotation = 60)
plt.grid(True)
plt.legend(loc = "upper left",prop = {"size" : 15})
plt.title("Events by gender for winter olympics")

plt.show()

# Olympic sports
* At the first Olympic Games, nine sports were contested. Since then, the number of sports contested at the Summer Olympic Games has gradually risen to twenty-eight on the program for 2000–2008. At the 2012 Summer Olympics, however, the number of sports fell back to twenty-six following an IOC decision in 2005 to remove baseball and softball from the Olympic program. These sports retain their status as Olympic sports with the possibility of a return to the Olympic program in future games. At the 121st IOC Session in Copenhagen on 9 October 2009, the IOC voted to reinstate both golf and rugby to the Olympic program, meaning that the number of sports to be contested in 2016 was once again 28
* Before 1924, when the first Winter Olympic Games were celebrated, sports held on ice, like figure skating and ice hockey, were held at the Summer Olympics. These two sports made their debuts at the 1908 and the 1920 Summer Olympics, respectively, but were permanently integrated in the Winter Olympics program as of the first edition.  The number of sports contested at the Winter Olympics has since been decreased to seven, comprising a total of fifteen disciplines

In [ ]:
sp_s = summer.groupby("Year")["Sport_category"].nunique().reset_index()
sp_w = winter.groupby("Year")["Sport_category"].nunique().reset_index()

plt.figure(figsize=(13,15))

plt.subplot(211)
ax = sns.pointplot(sp_s["Year"],sp_s["Sport_category"],color = "w")
ax.set_facecolor("b")
plt.xticks(rotation = 60)
plt.grid(True,alpha= .2)
plt.title("Sports for summer olympics")
plt.yticks(np.arange(8,30,2))
plt.ylabel("count")

plt.subplot(212)
ax = sns.pointplot(sp_w["Year"],sp_w["Sport_category"],color = "w")
ax.set_facecolor("b")
plt.xticks(rotation = 60)
plt.grid(True,alpha= .2)
plt.title("Sports for winter olympics")
plt.yticks(np.arange(3,9,1))
plt.ylabel("count")

plt.show()

## Summer olympics events for sports
The figures in each cell indicate the number of events for each sport contested at the respective Games; 

In [ ]:
#The figures in each cell indicate the number of events for each sport contested at the respective Games; 
sum_sp = pd.pivot_table(index="Sport",columns="Year",data=summer,
                        values="Event",aggfunc="nunique")
sum_sp = sum_sp.fillna(0)
plt.figure(figsize=(12,15))
sns.heatmap(sum_sp,linewidth=1,annot=True,cmap="rainbow",linecolor="k")
plt.title("Summer olympics events for sports",color="b")
plt.show()

## Winter olympics events for sports
The figures in each cell indicate the number of events for each sport contested at the respective Games; 

In [ ]:
win_sp = pd.pivot_table(index="Sport",columns="Year",data=winter,
                        values="Event",aggfunc="nunique")
win_sp = win_sp.fillna(0)
plt.figure(figsize=(12,10))
sns.heatmap(win_sp,linewidth=1,annot=True,cmap="rainbow",linecolor="k")
plt.title("Winter olympics events for sports",color="b")
plt.xticks(rotation = 90)
plt.show()

## Discontinued sports in olympics
* The following sports (or disciplines of a sport) make up the discontinued  Olympic Games official program.The discontinued sports were previously part of the Olympic Games program as official sports, but are no longer on the current program.

In [ ]:
disc_s = pd.pivot_table(data=summer,index="Sport",
                        columns="Year",values="Event",aggfunc="nunique")
print("Discontinued olympic summer sports  : \n",
      disc_s[disc_s[2016].isnull()].index.values)
disc_w = pd.pivot_table(index="Sport",columns="Year",
                        data=winter,values="Event",aggfunc="nunique")
print("\nDiscontinued olympic winter sports  : \n",
      disc_w[disc_w[2014].isnull()].index.values)

## Classification of Olympic sports for revenue share
* Summer Olympic sports are divided into categories based on popularity, gauged by: television viewing figures (40%), Internet popularity (20%), public surveys (15%), ticket requests (10%), press coverage (10%), and number of national federations (5%). The category determines the share the sport's International Federation receives of Olympic revenue.

* The current categories, as of 2013, are as follows, with the pre-2013 categorizations also being available.Category A represents the most popular sports; category E lists either the sports that are the least popular or that are new to the Olympics (golf and rugby).

In [ ]:

categories = pd.DataFrame({"CATEGORY" : ["A","B","C","D","E"],
                             "Individual Sports" :["Aquatics , Athletics ,Gymnastic",
                                                "Cycling , Tennis ",
                                                "Archery , Badminton , Boxing ,Judo , Rowing , Shooting ,Table Tennis ,Weightlifting",
                                                "Canoeing , Equestrianism , Sailing , Fencing , Taekwondo , Triathlon , Wrestling" ,
                                                "Modern Pentathlon , Golf"],
                             "Team Sport" : [" - ",
                                            "Basketball , Football , Volley",
                                            " - ",
                                            "Handball , Hockey ",
                                            "Rugby Sevens "]
                            })

categories_ind  = categories[["CATEGORY","Individual Sports"]]
categories_team = categories[["CATEGORY","Team Sport"]]

tab  = ff.create_table(categories_ind,height_constant=30)
tab1 = ff.create_table(categories_team,height_constant=30)
py.iplot(tab)
py.iplot(tab1)

In [ ]:
## Data Manipulation
#Creating new features

cat_A = ['Aquatics' , 'Athletics' ,'Gymnastic']
cat_B = ['Cycling' , 'Tennis','Basketball' , 'Football' , 'Volley']
cat_C = ['Archery' , 'Badminton' , 'Boxing' ,'Judo' ,
         'Rowing' , 'Shooting' ,'Table Tennis' ,'Weightlifting']
cat_D = ['Canoeing' , 'Equestrianism' , 'Sailing' ,'Fencing' ,
         'Taekwondo' , 'Triathlon' , 'Wrestling','Handball' , 'Hockey' ]
cat_E = ['Modern Pentathlon' , 'Golf',
        'Rugby Sevens' ]

#Gouping sports to categories

def rev_category(summer) :
    
    cat_A = ['Aquatics' , 'Athletics' ,'Gymnastic']
    for i in cat_A :
        if summer["Sport_category"] == i :
            return "Category A"
    
    cat_B = ['Cycling' , 'Tennis','Basketball' , 'Football' , 'Volley']
    for i in cat_B :
        if summer["Sport_category"] == i :
            return "Category B"
    
    cat_C = ['Archery' , 'Badminton' , 'Boxing' ,'Judo' ,
             'Rowing' , 'Shooting' ,'Table Tennis' ,'Weightlifting']
    for i in cat_C :
        if summer["Sport_category"] == i :
            return "Category C"
              
    cat_D = ['Canoeing' , 'Equestrianism' , 'Sailing' ,'Fencing' ,
             'Taekwondo' , 'Triathlon' , 'Wrestling','Handball' , 'Hockey' ]
    for i in cat_D :
        if summer["Sport_category"] == i :
            return "Category D"
    
    cat_E = ['Modern Pentathlon' , 'Golf','Rugby Sevens' ]
    for i in cat_E :
        if summer["Sport_category"] == i :
            return "Category E"
    
    else :
        return "Discontinued"
    
#Creating new column of category for respective sport
summer["revenue_category"]  = summer.apply(lambda summer:rev_category(summer),axis = 1)

#Individual sports
individual_sports = ['Aquatics' , 'Athletics' ,'Gymnastic','Cycling' , 'Tennis',
                     'Archery' , 'Badminton' , 'Boxing' ,'Judo' ,'Rowing' , 'Shooting' ,
                     'Table Tennis' ,'Weightlifting','Canoeing' , 'Equestrianism' , 
                     'Sailing' ,'Fencing' ,'Taekwondo' , 'Triathlon' , 'Wrestling',
                     'Modern Pentathlon' , 'Golf'  ,'Aeronautics','Art Competitions',
                     'Jeu De Paume','Motorboating','Racquets','Skating','Skiing'
                    ]
#team_sports
team_sports       = ['Basketball' , 'Football' , 'Volley',
                     'Handball' , 'Hockey','Rugby Sevens','Baseball','Basque Pelota',
                     'Cricket','Ice Hockey','Rugby','Croquet','Lacrosse', 'Polo',
                     'Roque','Tug-Of-War','Softball'
                    ]
#discontinued sports
discontinued     = ['Aeronautics', 'Art Competitions', 'Baseball', 'Basque Pelota',
                    'Cricket', 'Croquet', 'Ice Hockey', 'Jeu De Paume', 'Lacrosse',
                    'Motorboating', 'Polo', 'Racquets', 'Roque', 'Rugby', 'Skating',
                    'Skiing', 'Softball', 'Tug-Of-War']


def sport_type(summer) :
    for i in individual_sports :
        if summer["Sport_category"] == i :
            return "Individual Sport"
    for i in team_sports :
        if summer["Sport_category"] == i :
            return "Team Sport"

def discontinued_sport(summer) :
    for i in discontinued :
        if summer["Sport_category"] == i :
            return 1 
    else :
        return 0

#Creating new column for sport type (indiviaual or team)
summer["Sport_type"] = summer.apply(lambda summer:sport_type(summer) , axis = 1)

#Creating new column for discontinued sport  (1,0)
summer["discontinued"] = summer.apply(lambda summer:discontinued_sport(summer) , axis = 1)


## Percentage of Sports in each revenue category

In [ ]:
cat_sp_count = summer.groupby("revenue_category")["Sport_category"].nunique().reset_index()
cat_sp_sport = summer.groupby("revenue_category")["Sport_category"].unique().reset_index()
cat_sp_sport = cat_sp_sport.rename(columns = {"Sport_category" : "Sport"} )
cat_sp = cat_sp_count.merge(cat_sp_sport,left_on="revenue_category",
                            right_on="revenue_category",how ="left")

cat_ev_count = summer.groupby("revenue_category")["Event"].nunique().reset_index()
cat_ev_sport = summer.groupby("revenue_category")["Event"].unique().reset_index()
cat_ev_sport = cat_ev_sport.rename(columns = {"Event" : "Sport"} )
cat_ev = cat_ev_count.merge(cat_sp_sport,left_on="revenue_category",
                            right_on="revenue_category",how ="left")
cat_ev


labels = cat_sp["revenue_category"].tolist()[:5]
values = cat_sp["Sport_category"].tolist()[:5]
text   = cat_sp["Sport"].tolist()

labels1 = cat_ev["revenue_category"].tolist()[:5]
values1 = cat_ev["Event"].tolist()[:5]
text1   = cat_ev["Sport"].tolist()

trace = go.Pie(labels=labels,values=values,
               hoverinfo="label+percent",
               domain = dict(x = [0,.48]),
               hole = .45,
               marker=dict(line=dict(color='#000000', width=1)
                          )
              )
trace1 = go.Pie(labels=labels1,values=values1,
               hoverinfo="label+percent",
                hole = .45,
                domain = dict(x = [.54,1]),
               marker=dict(line=dict(color='#000000', width=1)
                          )
              )

data = [trace,trace1]

layout = go.Layout(dict(title = "Percentage of Sports and events in each revenue category",
                        annotations = [
                                 dict(text = "Sports",
                                      showarrow = False, 
                                      font = dict(size = 15),
                                      x = .2,
                                      y = .5,
                                     ),
                                 dict(text = "Events",
                                      showarrow = False,
                                      font = dict(size = 15),
                                      x = .81,
                                      y = .5,
                                     )
                             ]
                       )
                  )

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)


## Individual and team (events and sports) in each revenue category

In [ ]:
rev_sp = summer.groupby(["revenue_category","Sport_type"])[["Sport"]].nunique().reset_index()
rev_ev = summer.groupby(["revenue_category","Sport_type"])[["Event"]].nunique().reset_index()

plt.figure(figsize=(12,14))
plt.subplot(211)
sns.barplot("revenue_category","Sport",
            data=rev_sp,hue="Sport_type",
            linewidth = 1,edgecolor = "k"*len(rev_sp),
            palette = ["r","grey"],
           )
plt.grid(True)
plt.legend(loc = "upper left",prop= {"size" : 14})
plt.title("Individual and team sport in each revenue category",color="b")
plt.ylabel("count")

plt.subplot(212)
sns.barplot("revenue_category","Event",
            data=rev_ev,hue="Sport_type",
            linewidth = 1,edgecolor = "k"*len(rev_ev),
            palette = ["r","grey"],
           )
plt.grid(True)
plt.legend(loc = "upper right",prop= {"size" : 14})
plt.title("Individual and team events in each revenue category",color="b")
plt.ylabel("count")

plt.subplots_adjust(hspace = .3)
plt.show()

## Medals won in Revenue categories

In [ ]:
rev_med = pd.crosstab(summer["revenue_category"],summer["Medal"]).reset_index()[:5]

def bar(col,color) : 
    t = go.Bar(x = rev_med["revenue_category"],y=rev_med[col],
                marker = dict(color = color,
                              line  = dict(width = 1)
                             ),
                name = col
               )
    return t

t1 = bar("Gold","Gold")
t2 = bar("Silver","Silver")
t3 = bar("Bronze","#FF9900")

dat = [t1,t2,t3]

layout = go.Layout(dict(title ="Medals won in Revenue categories",
                        barmode = "stack",
                        xaxis = dict(title = "Revenu category",
                                     showgrid = True,
                                    ),
                        yaxis = dict(title = "Count",
                                     showgrid = True,
                                    )
                       )
                  )

fig  = go.Figure(data=dat,layout=layout)
py.iplot(fig)

## Olympic medals by sport.(Summer and winter olympics)

In [ ]:
spt_med = events.groupby("Sport")["Gold", 'Silver', 'Bronze',"Total"].sum().reset_index()
spt_med = spt_med.sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending =False)
spt_med

t = go.Table(header=dict(values = ['Sport','Gold', 'Silver', 'Bronze','Total'],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver",
                                               "#CC6633","royalblue"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [spt_med.Sport,spt_med.Gold,spt_med.Silver,
                                     spt_med.Bronze,spt_med.Total],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

layout = go.Layout(dict(title = "Olympic medals by sport.(Summer and winter olympics)"))

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

t1 = go.Bar(x = spt_med["Sport"],
            y = spt_med["Total"],
            marker = dict(color = "royalblue",
                          line  = dict(width = 1)
                         ),
            name = "Total Medals"
           )

dat = [t1]

lay = go.Layout(dict(title = "Olympic medals by sport." ,
                     margin = dict(b = 150),
                    xaxis  = dict(showgrid = True,
                                 ),
                    yaxis  = dict(showgrid = True,
                                  title = "Medal count",
                                 ),
                    plot_bgcolor = "white",
                     )
                )
    
figure = go.Figure(data=dat,layout=lay)
py.iplot(figure)


## Highest Multiple Olympic medalists ( Summer and Winter olympics)
* list of multiple Olympic medalists, i.e. those athletes who have won multiple Olympic medals at either the Summer Olympic Games or the Winter Olympic Games.

* American swimmer Michael Phelps is the most decorated Olympian, having won a total of 28 medals in both individual and 
team events (23 gold, 3 silver, and 2 bronze). 

* Top 100 athletes with highest total medals.

In [ ]:
med = pd.crosstab(events["Name"],events["Medal"]).reset_index()
med["Total"] = med['Gold'] + med['Silver'] + med['Bronze']
med_tp = med.sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending =False)[:100]

#Merge country
cou_lst = med_tp["Name"].tolist()
cou_lst = events[events["Name"].isin(cou_lst)][["Name","region"]]
cou_lst = cou_lst.drop_duplicates(subset=["Name","region"])
med_tp  = med_tp.merge(cou_lst,left_on="Name",
                       right_on="Name",how = "left").drop_duplicates("Name")
med_tp  = med_tp.rename(columns = {"region" : "Country"})

t = go.Table(header=dict(values = ['Name','Gold', 'Silver', 'Bronze','Total',"Country"],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver",
                                               "#CC6633","royalblue","#CCFF66"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [med_tp.Name,med_tp.Gold,med_tp.Silver,
                                     med_tp.Bronze,med_tp.Total,med_tp.Country],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

layout = go.Layout(dict(title = "Top 100 athletes with highest total medals.(Summer and winter olympics)"))

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

#top 10

def plot(dataframe,title):
    
    trace = go.Bar(y = dataframe["Name"],
                   x = dataframe["Gold"],
                   orientation="h",
                   marker = dict(color = "Gold",
                                 line  = dict(width = 1)
                                ),
                   name = "GOLD"
                  )
    trace1 = go.Bar(y = dataframe["Name"],
                   x = dataframe["Silver"],
                   orientation="h",
                   marker = dict(color = "Silver",
                                 line  = dict(width = 1)
                                ),
                    name = "SILVER"
                  )
    trace2 = go.Bar(y = dataframe["Name"],
                   x = dataframe["Bronze"],
                   orientation="h",
                   marker = dict(color = "#CC6633",
                                 line  = dict(width = 1)
                                ),
                    name  = "BRONZE"
                  )
    
    
    dat = [trace,trace1,trace2]
    
    lay = go.Layout(dict(title= title,
                         margin = dict(r = 100,l =300),
                         xaxis  = dict(showgrid = True),
                         yaxis  = dict(showgrid = True),
                         plot_bgcolor = "white",
                         height = 650
                        )
                   )
    
    figure = go.Figure(data=dat,layout=lay)
    py.iplot(figure)

plot(med_tp[:10].sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending = True),
     "Top 10 Athletes with highest medals overall(summer and winter olympics)")

## Highest Multiple Olympic medalists in  Summer olympics

In [ ]:
med_s = pd.crosstab(summer["Name"],summer["Medal"]).reset_index()
med_s["Total"] = med_s['Gold'] + med_s['Silver'] + med_s['Bronze']
med_tp_s = med_s.sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending =False)[:100]

#Merge country
cou_lst_s = med_tp_s["Name"].tolist()
cou_lst_s = summer[summer["Name"].isin(cou_lst_s)][["Name","region"]]
cou_lst_s = cou_lst_s.drop_duplicates(subset=["Name","region"])
med_tp_s  = med_tp_s.merge(cou_lst_s,left_on="Name",right_on="Name",how = "left").drop_duplicates("Name")
med_tp_s  = med_tp_s.rename(columns = {"region" : "Country"})
med_tp_s

t = go.Table(header=dict(values = ['Name','Gold', 'Silver', 'Bronze','Total',"Country"],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver","#CC6633",
                                               "royalblue","#CCFF66"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [med_tp_s.Name,med_tp_s.Gold,med_tp_s.Silver,
                                     med_tp_s.Bronze,med_tp_s.Total,med_tp_s.Country],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

l = go.Layout(dict(title = "Top 100 athletes with highest total medals in Summer olympics"))
f = go.Figure(data=data,layout=l)
py.iplot(f)


#top 10
plot(med_tp_s[:10].sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending = True),
     "Top 10 Athletes with highest medals  in summer olympics")

## Highest Multiple Olympic medalists in  Winter olympics

In [ ]:
med_w = pd.crosstab(winter["Name"],winter["Medal"]).reset_index()
med_w["Total"] = med_w['Gold'] + med_w['Silver'] + med_w['Bronze']
med_tp_w = med_w.sort_values(by = ["Total","Gold"],ascending =False)[:100]

#Merge country
cou_lst_w = med_tp_w["Name"].tolist()
cou_lst_w = winter[winter["Name"].isin(cou_lst_w)][["Name","region"]]
cou_lst_w = cou_lst_w.drop_duplicates(subset=["Name","region"])
med_tp_w  = med_tp_w.merge(cou_lst_w,left_on="Name",
                           right_on="Name",how = "left").drop_duplicates("Name")
med_tp_w  = med_tp_w.rename(columns = {"region" : "Country"})
med_tp_w

t = go.Table(header=dict(values = ['Name','Gold', 'Silver', 'Bronze','Total',"Country"],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver",
                                               "#CC6633","royalblue","#CCFF66"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [med_tp_w.Name,med_tp_w.Gold,med_tp_w.Silver,
                                     med_tp_w.Bronze,med_tp_w.Total,med_tp_w.Country],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

l = go.Layout(dict(title = "Top 100 athletes with highest total medals in Winter olympics"))
f = go.Figure(data=data,layout=l)
py.iplot(f)

#top10
plot(med_tp_w[:10].sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending = True),
     "Top 10 Athletes with highest medals in  winter olympics")

## Highest Female Olympic medalists

In [ ]:
female = events[events["Sex"] == "FEMALE"]
med_f  = pd.crosstab(female["Name"],female["Medal"]).reset_index()
med_f["Total"]  = med_f["Gold"] + med_f["Silver"] + med_f["Bronze"]
med_tp_f  = med_f.sort_values(by = ["Total","Gold"],ascending= False)[:100]
med_tp_f

#Merge country
cou_lst_f = med_tp_f["Name"].tolist()
cou_lst_f = female[female["Name"].isin(cou_lst_f)][["Name","region"]]
cou_lst_f = cou_lst_f.drop_duplicates(subset=["Name","region"])
med_tp_f  = med_tp_f.merge(cou_lst_f,left_on="Name",right_on="Name",
                           how = "left").drop_duplicates("Name")
med_tp_f  = med_tp_f.rename(columns = {"region" : "Country"})
med_tp_f

t = go.Table(header=dict(values = ['Name','Gold', 'Silver', 'Bronze','Total',"Country"],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver",
                                               "#CC6633","royalblue","#CCFF66"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [med_tp_f.Name,med_tp_f.Gold,med_tp_f.Silver,
                                     med_tp_f.Bronze,med_tp_f.Total,med_tp_f.Country],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

l = go.Layout(dict(title = "Top 100 Female athletes with highest total medals in olympics"))
f = go.Figure(data=data,layout=l)
py.iplot(f)


# top 10
plot(med_tp_f[:10].sort_values(by = ["Total","Gold", 'Silver', 'Bronze'],ascending = True),
     "Top 10 Female Athletes with highest medals in olympics")

# Medalist with most medals by Olympiad

In [ ]:
yearly_tp = events.groupby(["Year","Season","Name"])[["Medal","Gold","Silver","Bronze"]].sum().reset_index()
yearly_tp["Total"] = yearly_tp["Gold"] + yearly_tp["Silver"] +yearly_tp["Bronze"]
yearly_tp = yearly_tp.sort_values(by = ["Year","Total","Gold","Silver","Bronze"],ascending =False)
yearly_tp = yearly_tp.drop_duplicates(subset=["Year","Season"],keep="first")
yearly_tp

t = go.Table(header=dict(values = ["Year","Season",'Name','Gold', 'Silver', 'Bronze','Total'],
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","#CCFF66","#FF9999",
                                               "Gold","Silver","#CC6633","royalblue",]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [yearly_tp.Year,yearly_tp.Season,yearly_tp.Name,
                                     yearly_tp.Gold,yearly_tp.Silver,yearly_tp.Bronze,
                                     yearly_tp.Total],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

l = go.Layout(dict(title = "Medalist with most medals by Olympiad"))
f = go.Figure(data=data,layout=l)
py.iplot(f)


# Countries  with medals
* Medals include every individual medal in team sports.

In [ ]:
cou_med = events.groupby("region")["Gold","Silver","Bronze"].sum().reset_index()
cou_med["Total"] = cou_med["Gold"] + cou_med["Silver"] + cou_med["Bronze"]
cou_med = cou_med.sort_values(by = ["Total","Gold","Silver","Bronze"],ascending = False)
cou_med = cou_med.rename(columns={"region" : "Country"})

t = go.Table(header=dict(values = cou_med.columns.tolist(),
                         line = dict(color = 'black'),
                         align = ['left','center'],
                         fill  = dict(color = [ "#66CCFF","Gold","Silver","#CC6633","#CCFF66"]),
                         font = dict(color = 'black', size = 14)
                        ),
             cells  = dict(values = [cou_med.Country,cou_med.Gold,cou_med.Silver,
                                     cou_med.Bronze,cou_med.Total],
                           line = dict(color = '#506784'),
                           align = ['left','center'],
                           fill  = dict(color = "#F5F8FF"),
                          )
            )
data = [t]

l = go.Layout(dict(title = "Countries with most Medals(Summer and winter olympics)"))
f = go.Figure(data=data,layout=l)
py.iplot(f)


In [ ]:
clr = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]]

trace = go.Choropleth(locations = cou_med["Country"],
                      locationmode = "country names",
                      z = cou_med["Total"],
                      colorscale = "Viridis",
                      autocolorscale = False,
                      reversescale = True,
                      marker = dict(line = dict(width = .5,color = "black")),
                      colorbar = dict(title = "MEDALS COUNT ",
                                      titleside = "right")
                     )

data = [trace]

layout = go.Layout(dict(title = "Countries  with medals( Summer and winter olympics)",
                        geo = dict(showframe = True,
                                   showocean = True,
                                   showcountries = True,
                                   bgcolor = "white",
                                   showcoastlines = False,
                                   projection = dict(type = 'equirectangular')
                                  ),
                       )
                  )

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Countries without medals

In [ ]:
G = '\033[92m'
print (G + UNDERLINE+BOLD+"Countries without medals :")
np.array(cou_med[cou_med["Total"] == 0]["Country"])

In [ ]:
no_med = cou_med[cou_med["Total"] == 0]
no_med
trace = go.Choropleth(locations = no_med["Country"],
                      showscale=False,
                      locationmode = "country names",
                      z = no_med["Total"],
                      colorscale = "Hot",
                      reversescale = True,
                      autocolorscale = False,
                      hoverinfo = "location",
                      marker = dict(line = dict(width = 1)
                                   ),
                     )

data = [trace]

        
layout = go.Layout(dict(title = "Countries  without Medals",
                        geo = dict(showframe = True,
                                   showocean = True,
                                   showcountries = True,
                                   bgcolor = "white",
                                   showcoastlines = False,
                                   projection = dict(type = 'equirectangular')
                                  ),
                       )
                  )

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

# Boycotts
Greece, Australia, France, and United Kingdom are the only countries to be represented at every Olympic Games since their inception in 1896. While countries sometimes miss an Olympics due to a lack of qualified athletes, some choose to boycott a celebration of the Games for various reasons.

In [ ]:
boycott_nations = pd.read_csv(r"../input/olympic-boycott-nations/boycott.csv",
                              encoding='windows-1252')
boycott_nations["host_city"] = boycott_nations["host_city"].str.replace(" ","Berlin")
yr_by = boycott_nations.groupby("year")["boycott_countries"].nunique().reset_index()


plt.figure(figsize=(10,6))
sns.barplot(y =  yr_by["boycott_countries"],x = yr_by["year"] ,
             palette = "husl",linewidth = 1,
             edgecolor ="k"*len(yr_by))

plt.xticks(size = 20)
plt.xlabel("year")
plt.ylabel("Number of countries")
plt.title("Boycott nations by years")
plt.grid(True)

## Boycott nations for year 1956
There were three boycotts of the 1956 Melbourne Olympics: the Netherlands, Spain, and Switzerland refused to attend because of the repression of the Hungarian uprising by the Soviet Union, but did send an equestrian delegation to Stockholm; Cambodia, Egypt, Iraq, and Lebanon boycotted the Games because of the Suez Crisis; and China (the "People's Republic of China") boycotted the Games because Taiwan was allowed to compete in the Games as the "Republic of China"

In [ ]:
boycott_nations
boycott_nations["count"] = 0

def yearly_boycott(year) :
    
    x = boycott_nations[boycott_nations["year"] == year][["boycott_countries","count"]]
    tracer = go.Choropleth(locationmode = "country names",
                           locations    = x["boycott_countries"],
                           z = x["count"],
                           showscale = False,
                           reversescale = False,
                           colorscale = "Jet",
                           hoverinfo = "location",
                          )
    return tracer

def boycott_layout(year) :
    lay = go.Layout(dict(title = "Boycott nations for year : " + str(year),
                         geo = dict(showframe = True,
                                    showocean = True,
                                    showcoastlines = True,
                                    projection = dict(type = 'equirectangular'),
                                    showcountries = True,
                                  ),
                        )
                   )
    return lay

trace    = yearly_boycott(1956)
layout   = boycott_layout(1956)
data     = [trace]

fig      = go.Figure(data=data,layout=layout)

py.iplot(fig)

## Boycott nations for year 1976
1976 a large number of African countries threatened the IOC with a boycott to force them to ban South Africa and Rhodesia, because of their segregationist regimes. New Zealand was also one of the African boycott targets, because its national rugby union team had toured apartheid-ruled South Africa. The IOC conceded in the first two cases, but refused to ban New Zealand on the grounds that rugby was not an Olympic sport. Fulfilling their threat, twenty African countries were joined by Guyana and Iraq in a withdrawal from the Montreal Games, after a few of their athletes had already competed.

In [ ]:
trace    = yearly_boycott(1976)
layout   = boycott_layout(1976)
data     = [trace]

fig      = go.Figure(data=data,layout=layout)

py.iplot(fig)

## Boycott nations for years 1980 & 1984
In 1980 and 1984, the Cold War opponents boycotted each other's Games. The United States and sixty-five other countries boycotted the Moscow Olympics in 1980 because of the Soviet invasion of Afghanistan. This boycott reduced the number of nations participating to 80, the lowest number since 1956.The Soviet Union and 15 other nations countered by boycotting the Los Angeles Olympics of 1984, contending that they could not guarantee the safety of their athletes. Although a boycott led by the Soviet Union depleted the field in certain sports, 140 National Olympic Committees took part, which was a record at the time.The fact that Romania, a Warsaw Pact country, opted to compete despite Soviet demands led to a warm reception of the Romanian team by the United States. When the Romanian athletes entered during the opening ceremonies, they received a standing ovation from the spectators, which comprised mostly U.S. citizens. The boycotting nations of the Eastern Bloc staged their own alternate event, the Friendship Games, in July and August

In [ ]:
trace1    = yearly_boycott(1980)
layout1   = boycott_layout(1980)
data1     = [trace1]

fig1      = go.Figure(data=data1,layout=layout1)


trace2    = yearly_boycott(1984)
layout2   = boycott_layout(1984)
data2     = [trace2]

fig2      = go.Figure(data=data2,layout=layout2)

py.iplot(fig1)
py.iplot(fig2)

# 1906 Intercalated Games
The Intercalated Olympic Games were to be a series of International Olympic Games halfway between what is now known as the Games of the Olympiad. This proposed series of games, intercalated in the Olympic Games cycle, was to always be held in 

Athens, and were to have equal status with the international games. However, the only such games were held in 1906.
The Greeks were unable to keep the schedule for 1910. On the one hand problems in the Balkans made things difficult, but on the other, the modern Greeks found out the ancient Greeks were right: A two-year interval was too short. Where there had been a gap of six years before Athens 1906 (because of the almost all-American nature of the 1904 St Louis games), a gap of two years after London 1908 did not leave people enough time to prepare.

With Athens 1910 a failure, the faith in Athens diminished, and as a result Athens 1914 got even less support. And then World War I started, and any further Intercalated Games had to wait until after the war. But after the war was over it had been more than a decade since Athens 1906, and the idea of Intercalated Games was given up entirely.

In [ ]:
#Intercalated Games
intercarlated = events[events["Year"] == 1906]
athlete_partc = intercarlated.groupby("region")["ID"].nunique().reset_index()
athlete_partc["x"] = 0
meds_ic = intercarlated.groupby("region")[["Gold","Silver","Bronze","Total"]].sum().reset_index()
athlete_partc_ic = athlete_partc.merge(meds_ic,left_on = "region",right_on= "region" ,how = "left")
athlete_partc


host_ic = pd.DataFrame({"country"  : ["Greece"],
                        "city"     : ["Athens"],
                        "latitude": [37.983333],
                        "longitude": [23.733334],
                        "year"    : 1906
                        }
                      )

trace = go.Choropleth(locationmode = "country names",
                      locations=athlete_partc["region"] ,
                      z = athlete_partc["x"],
                      showscale  =  False,
                      colorscale = "Portland",
                      reversescale = False,
                      hoverinfo = "location",
                      geo = "geo3"
                     )

trace2 = go.Scattergeo(lat = host_ic["latitude"],
                       lon = host_ic["longitude"],
                       geo = "geo3",
                       marker  = dict(size = 10,
                                      color = "red",
                                      line  = dict(width = 1,
                                                   color = "black"
                                                  ) 
                                     ),
                       name = "Athens",
                       hoverinfo = "name"
                      )

lay = go.Layout(dict(title = "Participants of the 1906 Games",
                         geo3 = dict(showframe = True,
                                     showocean = True,
                                     showcoastlines = True,
                                     projection = dict(type = 'equirectangular'),
                                     showcountries = True,
                                  ),
                        )
                   )


fig = go.Figure(data=[trace,trace2],layout=lay)
py.iplot(fig)

###
trace2 = go.Scatter(x = athlete_partc_ic["region"],
                    y = athlete_partc_ic["ID"],
                    mode = "lines+markers",
                    marker  = dict(color = "red",
                                   line  = dict(width = 1,
                                                color = "grey") 
                                  ),
                    name  = "Athletes"
                   )
trace3 = go.Scatter(x = athlete_partc_ic["region"],
                    y = athlete_partc_ic["Total"],
                    mode = "lines+markers",
                    marker  = dict(color = "blue",
                                   line  = dict(width = 1,
                                                color = "grey") 
                                  ),
                    name  = "Medals"
                   )

lay1 = go.Layout(dict(title = "Athletes participation and medals achieved by countries - 1906 intercarlated Games",
                      xaxis = dict(tickangle = 90,
                                   mirror = "ticks",
                                   showgrid = True,
                                   zeroline = True,
                                  ),
                      yaxis = dict(showgrid =True)
                     )
                )
fig1 = go.Figure(data=[trace2,trace3],layout=lay1)
py.iplot(fig1)

## Yearly top nations with highest medals in olympics

In [ ]:
yearly_top_n = events.groupby(["Year","Season","region"])["Gold","Silver","Bronze"].sum().reset_index()
yearly_top_n["total"] = yearly_top_n["Gold"] + yearly_top_n["Silver"] + yearly_top_n["Bronze"]
yearly_top_n = yearly_top_n.sort_values(by = ["total","Gold","Silver","Bronze"],ascending  = False)
yearly_top_n = yearly_top_n.drop_duplicates(subset=["Year","Season"],keep="first").sort_values(by = "Year",ascending = False)

yearly_top_ns = yearly_top_n[yearly_top_n["Season"] == "Summer"]
yearly_top_nw = yearly_top_n[yearly_top_n["Season"] == "Winter"]
yearly_top_nw

#fuction for bubble plot
def plot_top_nations(df,country,ratio) : 
    tracer = go.Scatter(x = df[df["region"] == country]["Year"],
                        y = df[df["region"] == country]["total"],
                        mode  = "markers",
                        marker = dict(line = dict(width =1,color = "black"),
                                      size = df[df["region"] == country]["total"]/ratio,
                                     ),
                        name  = country 
                       )
    return tracer

def plot_layout(title) :
    layout = go.Layout(dict(title = title,
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "Year",
                                     zerolinewidth=1,
                                     ticklen=5,
                                     gridwidth=2
                                    ),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "Medal Count",
                                     zerolinewidth=1,
                                     ticklen=5,
                                     gridwidth=2
                                    ),
                        paper_bgcolor = "rgb(243,243,243)",
                        plot_bgcolor  = "rgb(243,243,243)",
                       )
                  )
    return layout

#plotting summer olympics
ts1 = plot_top_nations(yearly_top_ns,"USA",10)
ts2 = plot_top_nations(yearly_top_ns,"Russia",10)
ts3 = plot_top_nations(yearly_top_ns,'Germany',10)
ts4 = plot_top_nations(yearly_top_ns,'Sweden',10)
ts5 = plot_top_nations(yearly_top_ns,"UK",10)
ts6 = plot_top_nations(yearly_top_ns,'Greece',10)
ts7 = plot_top_nations(yearly_top_ns,'France',10)

dat1 = [ts1,ts2,ts3,ts4,ts5,ts6,ts7]
lay1 = plot_layout("Yearly top nations with highest medals in summer olympics")
fig1 = go.Figure(data=dat1,layout=lay1)

py.iplot(fig1)

#plotting winter olympics
tw1  = plot_top_nations(yearly_top_nw,"USA",2)
tw2  = plot_top_nations(yearly_top_nw,"Russia",2)
tw3  = plot_top_nations(yearly_top_nw,'Germany',2)
tw4  = plot_top_nations(yearly_top_nw,'Sweden',2)
tw5  = plot_top_nations(yearly_top_nw,"UK",2)
tw6  = plot_top_nations(yearly_top_nw,'Greece',2)
tw7  = plot_top_nations(yearly_top_nw,'France',2)
tw8  = plot_top_nations(yearly_top_nw,'Canada',2)
tw9  = plot_top_nations(yearly_top_nw,'Finland',2)
tw10 = plot_top_nations(yearly_top_nw,'Switzerland',2)
tw11 = plot_top_nations(yearly_top_nw,'Norway',2)

dat2 = [tw1,tw2,tw3,tw4,tw5,tw6,tw7,tw8,tw9,tw10,tw11]
lay2 = plot_layout("Yearly top nations with highest medals in winter olympics")
fig2 = go.Figure(data=dat2,layout=lay2)

py.iplot(fig2)

# Country wise analysis

In [ ]:

def country_wise_analysis(country_name) :
    
    data   = events[events["region"] == country_name].sort_values(by = "Year",
                                                                  ascending = False)
    s_data = data[data["Season"] == "Summer"]
    w_data = data[data["Season"] == "Winter"]
    
    RED = '\033[91m'
    PURPLE = '\033[95m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

    print (BLUE+UNDERLINE+BOLD+country_name+END)
    print ("--------------------")
    print ("Olympics appearances  : " , data["Games"].nunique(),"\n")
    
    print (RED+UNDERLINE+BOLD+ "Summer Olympics"+END)
    print ("--------------------")
    print ("Summer Olympics appearances : ", s_data["Year"].nunique())
    print ("Total Participants          : ", s_data["ID"].nunique())
    print ("Female Participants         : ", s_data[s_data["Sex"] == "FEMALE"]["ID"].nunique(),
           ", Percent : " ,
           np.around(s_data[s_data["Sex"] == "FEMALE"]["ID"].nunique()/s_data["ID"].nunique()*100,2),
           "%")
    print ("Male   Participants         : ", s_data[s_data["Sex"] == "MALE"]["ID"].nunique(),
           ", Percent : " ,
           np.around(s_data[s_data["Sex"] == "MALE"]["ID"].nunique()/s_data["ID"].nunique()*100,2),
           "%")
    print ("Competed Sports             : ",s_data["Sport"].nunique())
    print ("Competed Events             : ",s_data["Event"].nunique())
    
    female_events_summer = s_data[s_data["Sex"] == "FEMALE"]["Event"].unique().tolist()
    male_events_summer   = s_data[s_data["Sex"] == "MALE"]["Event"].unique().tolist()
    both_events_summer   = [x for x in male_events_summer if x in female_events_summer]
    print ("Male Events                 : ",len(male_events_summer))
    print ("Female Events               : ",len(female_events_summer))
    print ("Events  for both genders    : ",len(both_events_summer))
    print ("Total Medals                : ",s_data["Total"].sum() , "(medals include every individual medal in team sport)")
    print ("Gold                        : ",s_data["Gold"].sum())
    print ("Silver                      : ",s_data["Silver"].sum())
    print ("Bronze                      : ",s_data["Bronze"].sum())
    pop_spo_sum   = (s_data.groupby("Sport")["Total"].sum().reset_index().
                     sort_values(by = "Total",ascending =False))
    print ("Popular Sports              : " ,pop_spo_sum["Sport"][:3].tolist())
    pop_play_sum  = (s_data.groupby(["Name","Sport"])["Total","Gold","Silver","Bronze"].sum().reset_index().
                    sort_values(by = ["Total","Gold","Silver","Bronze"],ascending =False))
    print ("Most Medals  (Athlete)      : " ,pop_play_sum.loc[pop_play_sum["Total"].idxmax()]["Name"],
           ",Sport - ", pop_play_sum.loc[pop_play_sum["Total"].idxmax()]["Sport"])
    
    ###
        
    print ("\n",PURPLE+UNDERLINE+BOLD+ "Winter Olympics"+END)
    print ("--------------------")
    print ("Winter Olympics appearances : ", w_data["Year"].nunique())
    print ("Total Participants          : ", w_data["ID"].nunique())
    print ("Female Participants         : ", w_data[w_data["Sex"] == "FEMALE"]["ID"].nunique(),
           ", Percent : " ,
           np.around(w_data[w_data["Sex"] == "FEMALE"]["ID"].nunique()/w_data["ID"].nunique()*100,2),
           "%")
    print ("Male   Participants         : ", w_data[w_data["Sex"] == "MALE"]["ID"].nunique(),
           ", Percent : " ,
           np.around(w_data[w_data["Sex"] == "MALE"]["ID"].nunique()/w_data["ID"].nunique()*100,2),
           "%")
    print ("Competed Sports             : ",w_data["Sport"].nunique())
    print ("Competed Events             : ",w_data["Event"].nunique())
    
    female_events_winter = w_data[w_data["Sex"] == "FEMALE"]["Event"].unique().tolist()
    male_events_winter   = w_data[w_data["Sex"] == "MALE"]["Event"].unique().tolist()
    both_events_winter   = [x for x in male_events_winter if x in female_events_winter]
    print ("Male Events                 : ",len(male_events_winter))
    print ("Female Events               : ",len(female_events_winter))
    print ("Events  for both genders    : ",len(both_events_winter))
    print ("Total Medals                : ",w_data["Total"].sum() , "(medals include every individual medal in team sport)")
    print ("Gold                        : ",w_data["Gold"].sum())
    print ("Silver                      : ",w_data["Silver"].sum())
    print ("Bronze                      : ",w_data["Bronze"].sum())
    pop_spo_win   = (w_data.groupby("Sport")["Total"].sum().reset_index().
                     sort_values(by = "Total",ascending =False))
    print ("Popular Sports              : " ,pop_spo_win["Sport"][:3].tolist())
    pop_play_win  = (w_data.groupby(["Name","Sport"])["Total","Gold","Silver","Bronze"].sum().reset_index().
                    sort_values(by = ["Total","Gold","Silver","Bronze"],ascending =False))
    print ("Most Medals  (Athlete)      : " ,pop_play_win.loc[pop_play_win["Total"].idxmax()]["Name"],
           ",Sport - ", pop_play_win.loc[pop_play_win["Total"].idxmax()]["Sport"])

# India at the Olympics
India first participated at the Olympic Games in 1900, with a lone athlete (Norman Pritchard) winning two medals- both silver- in athletics. The nation first sent a team to the Summer Olympic Games in 1920, and has participated in every Summer Games since then. India has also competed at several Winter Olympic Games beginning in 1964.For a period of time, India national field hockey team was dominant in Olympic competition, winning eleven medals in twelve Olympics between 1920 and 1980. 

In [ ]:
country_wise_analysis("India")


# USA
The United States of America has sent athletes to every celebration of the modern Olympic Games with the exception of the 1980 Summer Olympics, during which it led a boycott. 

In [ ]:
country_wise_analysis("USA")

# Canada

In [ ]:
country_wise_analysis("Canada")

## Comparing Countries performance yearly
* Medals acieved by nations yearly

In [ ]:
#Function to compare two countries medal achievements over years
def country_compare(country1,country2) :   
    
    cn_lst = [country1,country2]
    df =  events[(events["region"].isin(cn_lst))]
    df =  df.groupby(["Year","Season","region"])["Gold","Silver","Bronze"].sum().reset_index()
    df["Total_medals"] = df["Gold"] + df["Silver"] + df["Bronze"]
    
    summer = df[df["Season"] == "Summer"]
    winter = df[df["Season"] == "Winter"] 
    
    def bar_compare(dataframe,aggregate,color) :
        tracer =     go.Bar(x = dataframe[dataframe["region"] == aggregate]["Year"],
                            y = dataframe[dataframe["region"] == aggregate]["Total_medals"],
                
                            marker = dict(line  = dict(width = 1),
                                          color = color,
                                         ),
                            name = aggregate,
                           )
        return tracer
    
    def layout_style(title) :
        layout = go.Layout(dict(title = title,
                                xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "Year",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                                yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "Medals",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                                paper_bgcolor = "rgb(243,243,243)",
                                plot_bgcolor  = "rgb(243,243,243)"
                               ),
                          )
        return layout
        
    trace1  = bar_compare(summer,country1,"orange")     
    trace2  = bar_compare(summer,country2,"grey")
    trace3  = bar_compare(winter,country1,"orange")
    trace4  = bar_compare(winter,country2,"grey")
    
    data1   = [trace1,trace2]
    data2   = [trace3,trace4]
    
    layout1 = layout_style(country1 + " & " + country2 + " performance in summer olympics" )
    layout2 = layout_style(country1 + " & " + country2 + " performance in winter olympics")
    
    fig1    = go.Figure(data = data1,layout = layout1)
    fig2    = go.Figure(data = data2,layout = layout2)
    
    py.iplot(fig1)
    py.iplot(fig2)

## USA & Russia

In [ ]:
country_compare("USA","Russia")

## USA & Germany

In [ ]:
country_compare("USA","Germany")

## "Yearly performance of countries and top players"

In [ ]:
def yearly_perf(year,season) :
    
    if season == "Summer" :
        dataframe = events[(events["Season"] == "Summer") & (events["Year"] == year)] 
    elif season == "Winter" :
        dataframe = events[(events["Season"] == "Winter") & (events["Year"] == year)]
        
    dataframe["Total"] = dataframe["Gold"] + dataframe["Silver"] + dataframe["Bronze"]
    
        
    choropleth_df = dataframe.groupby("region")["Gold","Silver","Bronze","Total"].sum().reset_index()
    choropleth_df = choropleth_df.sort_values(by = ["Total","Gold","Silver","Bronze"],ascending = False)
    
    top_players = dataframe.groupby("Name")["Gold","Silver","Bronze","Total"].sum().reset_index()
    top_players = top_players.sort_values(by = ["Total","Gold","Silver","Bronze"],ascending = False)
    
    hostcity = host[host["Year"] == str(year)]
    
    if season == "Summer" :
        hostcity = hostcity[hostcity["season"] == "SUMMER"]
    if season == "Winter" : 
        hostcity = hostcity[hostcity["season"] == "WINTER"]
    
    
    trace1 = go.Choropleth(z = choropleth_df["Total"],
                           locationmode="country names",
                           locations = choropleth_df["region"],
                           marker = dict(line = dict(width  = .5,
                                                     color  = "black")),
                           colorbar = dict(title = "MEDALS COUNT ",
                                           titleside = "right",
                                           thickness = 20,
                                           len = .4,
                                           x = -.1,
                                           y = .45,
                                          ),
                           geo = "geo3",
                           showscale = True,
                           name = "Medals",
                           hoverlabel = dict(bgcolor = "white"),
                          )
    
    trace2 = go.Bar(x = top_players["Name"][:5] , y = top_players["Gold"][:5],
                   marker = dict(color = "gold",
                                 line = dict(width = 1,
                                            color = "white")
                                ),
                   name = "gold",
                   hoverlabel = dict(bgcolor = "white")
                   )
                    
    trace3 = go.Bar(x = top_players["Name"][:5] , y = top_players["Silver"][:5],
                   marker = dict(color = "silver",
                                 line = dict(width = 1,
                                            color = "white")
                                ),
                                
                    name = "silver",
                    hoverlabel = dict(bgcolor = "white")
                   )
                    
    trace4 = go.Bar(x = top_players["Name"][:5] , y = top_players["Bronze"][:5],
                   marker = dict(color = "red",
                                 line = dict(width = 1,
                                            color = "white")
                                ),
                    name = "bronze",
                    hoverlabel = dict(bgcolor = "white")
                   )
    
    trace5 = go.Scattergeo(lat=hostcity["latitude"],lon = hostcity["longitude"],
                           geo="geo3",
                           mode = "markers",
                           marker = dict(size = 20,
                                         color = "red",
                                         line  = dict(width = 2,
                                                     color = "black"),
                                        ),
                           text = "Host city  : " + hostcity["City"],
                           name = "Host city",
                           hoverlabel = dict(bgcolor = "white"),
                           opacity = .9
                          )
    
    data = [trace1,trace2,trace3,trace4,trace5]
    
    layout = go.Layout(dict(plot_bgcolor  = "black",
                            paper_bgcolor = "black" ,
                            titlefont  = dict(size = 20,
                                              family = "Raleway"),
                            font = dict(color = "white"),
                            dragmode = "zoom",
                            geo3 = dict(domain = dict(x = [0,.55] , y = [0,.9]),
                                        showland   = True,
                                        showocean  = True,
                                        landcolor  = "white",
                                        bgcolor    = "black",
                                        projection = dict(type = "orthographic",
                                                          rotation = dict(lat = np.around(hostcity.reset_index().at[0,"latitude"],0),
                                                                          lon = np.around(hostcity.reset_index().at[0,"longitude"],0)
                                                                         )
                                                         ),
                                        showcountries = True,
                                       ),
                            margin = dict(r = 20,t = 25,
                                          b = 40,l = 60),
                            scene  = dict(domain = dict(x = [0.5,1],y = [0,0.55]),
                                          xaxis  = dict(gridcolor = "white"),
                                          yaxis  = dict(gridcolor = "white"),
                                         ),
                            showlegend = False,
                            xaxis = dict(anchor = "y" , 
                                         domain =  [0.6, 0.95] ),
                            yaxis = dict(anchor = "x",
                                         domain = [0.26, 0.95],
                                         showgrid = True,
                                        ),
                           title = "Yearly performance of countries and top players - " + str(year),
                           ),
                      )
    
    annotations = dict(text = "Year : " + str(year) + ", Season : " + season ,
                       showarrow = False,
                       xref = "paper",
                       yref = "paper",
                       x = 0,
                       y = 0,
                      )
    
    layout['annotations'] = [annotations]
    
    figure = go.Figure(data=data,layout=layout)
    DARKCYAN = '\033[36m'
    print (BLUE + BOLD + "Host Country                 : " , hostcity.reset_index().at[0,"Country"])
    print ("Host City                    : " , hostcity.reset_index().at[0,"City"])
    print ("Total regions participated   : " , dataframe["region"].nunique())
    print ("Total Participants           : " , dataframe["ID"].nunique())
    print ("Total Female Participants    : " , dataframe[dataframe["Sex"] == "FEMALE"]["ID"].nunique())
    print ("Total Male Participants      : " , dataframe[dataframe["Sex"] == "MALE"]["ID"].nunique())
    print ("Number of Sports             : " , dataframe["Sport"].nunique())
    print ("Number of Events             : " , dataframe["Event"].nunique())
    
    male_ev   = dataframe[dataframe["Sex"] == "MALE"]["Event"].unique().tolist()
    female_ev = dataframe[dataframe["Sex"] == "FEMALE"]["Event"].unique().tolist()
    both_ev   = [x for x in male_ev if x in female_ev]
    male_events   = [x for x in male_ev if x not in both_ev]
    female_events = [x for x in female_ev if x not in both_ev]
    print("Events for female athletes   : " , len(female_events))
    print("Events for male athletes     : " , len(male_events))
    print("Events for both genders      : " , len(both_ev))
    
    print("Youngest Participant  :  ",dataframe.loc[dataframe["Age"].idxmin()]["Name"],
      " , Age - " , (dataframe.loc[dataframe["Age"].idxmin()]["Age"]).astype(int),
      " , Country - " ,dataframe.loc[dataframe["Age"].idxmin()]["region"] )
    print("Oldest Participant    :  ",dataframe.loc[dataframe["Age"].idxmax()]["Name"],
      " , Age - " , (dataframe.loc[dataframe["Age"].idxmax()]["Age"]).astype(int),
      " , Country - " ,dataframe.loc[dataframe["Age"].idxmax()]["region"] )    
    med  = dataframe.groupby(["Name","Sport"])["Total"].sum().reset_index()
    print("Most Medals (Athlete) :  ", med.loc[med["Total"].idxmax()]["Name"],
          ", Sport - " ,med.loc[med["Total"].idxmax()]["Sport"])
    med_c = dataframe.groupby("region")["Total"].sum().reset_index()
    print("Most Medals (Country) :  ",med_c.loc[med_c["Total"].idxmax()]["region"])
    
    
    py.iplot(figure)
    

##  2016 Summer Olympics
The 2016 Summer Olympics officially known as the Games of the XXXI Olympiad and commonly known as Rio 2016, was an international multi-sport event that was held from 5 to 21 August 2016 in Rio de Janeiro, Brazil, with preliminary events in some sports beginning on 3 August. These were the first Olympic Games ever to be held in South America.

In [ ]:
yearly_perf(2016,"Summer")

## 2010 Winter Olympics
The 2010 Winter Olympics, officially known as the XXI Olympic Winter Games and commonly known as Vancouver 2010, informally the 21st Winter Olympics, was an international winter multi-sport event that was held from 12 to 28 February 2010 in Vancouver, British Columbia, Canada, with some events held in the surrounding suburbs of Richmond, West Vancouver and the University Endowment Lands, and in the nearby resort town of Whistler. 

In [ ]:
yearly_perf(2010,"Winter")

## 1996 Summer Olympics
The 1996 Summer Olympics, officially known as the Games of the XXVI Olympiad and unofficially referred to as the Centennial Olympic Games, was an international multi-sport event that was celebrated from July 19 to August 4, 1996, in Atlanta, Georgia, United States. A record 197 nations, all current IOC member nations, took part in the Games, fielding a total of 10,339 athletes. 

In [ ]:
yearly_perf(1996,"Summer")